In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame (
    [[8,8,1],[7,9,1],[6,10,0],[5,12,1],[4,10,0]],
    columns=['internships', 'profile_score', 'placed']
)

df

,internships,profile_score,placed
0,8,8,1
1,7,9,1
2,6,10,0
3,5,12,1
4,4,10,0


In [3]:
# layers will contain the size of each layer
def initialize(layers): # initialize all the weights to 1 and bias to 0
    parameters = {}

    for l in range (1, len(layers)):
        parameters['W'+str(l)] = np.ones((layers[l-1], layers[l]))
        parameters['b'+str(l)] = np.zeros((layers[l], 1))

    return parameters


In [4]:
def sigmoid(Z):
  return 1/(1+np.exp(-Z))

In [5]:
def layer_calc(A_prev, w, b):
    Z = np.dot(w.T, A_prev) + b
    return sigmoid(Z)

In [6]:
def forward_propagation(X, parameters):
    A = X
    L = len(parameters) // 2 # number of layers in the neural network

    for l in range(1, L+1):
        A_prev = A
        wl = parameters['W'+str(l)]
        bl = parameters['b'+str(l)]

        A = layer_calc(A_prev, wl, bl)

    y_hat = A[0][0]
    return y_hat, A_prev

In [7]:
def back_propogation(parameters, y, y_hat, A1, X): #updating the parameters
    learning_rate = 0.001
    parameters['W2'][0][0] = parameters['W2'][0][0] + (learning_rate * 2 * (y - y_hat)*A1[0][0])
    parameters['W2'][1][0] = parameters['W2'][1][0] + (learning_rate * 2 * (y - y_hat)*A1[1][0])
    parameters['b2'][0][0] = parameters['W2'][1][0] + (learning_rate * 2 * (y - y_hat))

    parameters['W1'][0][0] = parameters['W1'][0][0] + (learning_rate * 2 * (y - y_hat)*parameters['W2'][0][0]*X[0][0])
    parameters['W1'][0][1] = parameters['W1'][0][1] + (learning_rate * 2 * (y - y_hat)*parameters['W2'][0][0]*X[1][0])
    parameters['b1'][0][0] = parameters['b1'][0][0] + (learning_rate * 2 * (y - y_hat)*parameters['W2'][0][0])

    parameters['W1'][1][0] = parameters['W1'][1][0] + (learning_rate * 2 * (y - y_hat)*parameters['W2'][1][0]*X[0][0])
    parameters['W1'][1][1] = parameters['W1'][1][1] + (learning_rate * 2 * (y - y_hat)*parameters['W2'][1][0]*X[1][0])
    parameters['b1'][1][0] = parameters['b1'][1][0] + (learning_rate * 2 * (y - y_hat)*parameters['W2'][1][0])

In [8]:
parameters = initialize([2, 2, 1]) #network with 3 layers
epochs = 100

for i in range(1, epochs+1):
    loss = []

    for j in range(df.shape[0]): #iterating over all rows in the df
        X = df[['internships', 'profile_score']].values[j].reshape(2,1)
        y = df[['placed']].values[j][0]

        y_hat, A1 = forward_propagation(X, parameters)
        back_propogation(parameters, y, y_hat, A1, X)

        loss.append(-y*np.log(y_hat) - (1-y)*np.log(1-y_hat)) # Loss function is LogLoss and Activation Function is Sigmoid

    if(i%5==0):
        print(f"Epoch - {i}, Loss - {np.array(loss).mean()}")

print("Final Parameters: ", parameters)

Epoch - 5, Loss - 1.2332142081647435
Epoch - 10, Loss - 1.2149030262445208
Epoch - 15, Loss - 1.196705560263507
Epoch - 20, Loss - 1.1772367526361058
Epoch - 25, Loss - 1.1442117677402093
Epoch - 30, Loss - 1.0421172345883298
Epoch - 35, Loss - 0.9168754350956609
Epoch - 40, Loss - 0.8739879481790787
Epoch - 45, Loss - 0.8621971346290295
Epoch - 50, Loss - 0.8564478930077953
Epoch - 55, Loss - 0.8510491776724329
Epoch - 60, Loss - 0.844256084681956
Epoch - 65, Loss - 0.8348048431619519
Epoch - 70, Loss - 0.8215404885788293
Epoch - 75, Loss - 0.8042151050975681
Epoch - 80, Loss - 0.784552002161947
Epoch - 85, Loss - 0.7657400428580472
Epoch - 90, Loss - 0.7502066940406115
Epoch - 95, Loss - 0.7384792719876792
Epoch - 100, Loss - 0.7299227352123843
Final Parameters:  {'W1': array([[-0.0248405 , -1.29940446],
       [-0.07276898, -1.41786549]]), 'b1': array([[-0.2276903 ],
       [-0.23934172]]), 'W2': array([[0.76537822],
       [0.89429536]]), 'b2': array([[0.89279351]])}


# Using Keras

In [9]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Input

In [10]:
model = Sequential()

model.add(Input(shape=(2,)))
model.add(Dense(2, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

In [11]:
optimizer = tensorflow.keras.optimizers.legacy.Adam(learning_rate=0.01)
# optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer)

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2)                 6         
                                                                 
 dense_1 (Dense)             (None, 1)                 3         
                                                                 
Total params: 9 (36.00 Byte)
Trainable params: 9 (36.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
model.get_weights()

[array([[ 0.08305001, -0.79011786],
        [-0.05051255,  0.9971217 ]], dtype=float32),
 array([0., 0.], dtype=float32),
 array([[-0.4174615],
        [ 1.3361219]], dtype=float32),
 array([0.], dtype=float32)]

In [14]:
model.set_weights([
    np.array([[1,1],[1,1]]), np.array([0,0]), np.array([[1],[1]]), np.array([0])
])

In [15]:
model.fit(df[['internships', 'profile_score']].values, df['placed'], epochs=100, verbose=1, batch_size=1)

Epoch 1/100
5/5 [==============================] - 0s 966us/step - loss: 0.9275
Epoch 2/100
5/5 [==============================] - 0s 672us/step - loss: 0.8900
Epoch 3/100
5/5 [==============================] - 0s 647us/step - loss: 0.8722
Epoch 4/100
5/5 [==============================] - 0s 746us/step - loss: 0.8650
Epoch 5/100
5/5 [==============================] - 0s 631us/step - loss: 0.8369
Epoch 6/100
5/5 [==============================] - 0s 633us/step - loss: 0.8241
Epoch 7/100
5/5 [==============================] - 0s 600us/step - loss: 0.8067
Epoch 8/100
5/5 [==============================] - 0s 672us/step - loss: 0.7974
Epoch 9/100
5/5 [==============================] - 0s 568us/step - loss: 0.7903
Epoch 10/100
5/5 [==============================] - 0s 627us/step - loss: 0.7755
Epoch 11/100
5/5 [==============================] - 0s 584us/step - loss: 0.7608
Epoch 12/100
5/5 [==============================] - 0s 644us/step - loss: 0.7466
Epoch 13/100
5/5 [===================

In [16]:
model.get_weights()

[array([[ 0.5377929 ,  0.5377929 ],
        [-0.22082202, -0.22082202]], dtype=float32),
 array([-1.126851, -1.126851], dtype=float32),
 array([[0.83794653],
        [0.83794653]], dtype=float32),
 array([-0.3923642], dtype=float32)]